In [ ]:
#Classificador Gaussiano Friedman
import numpy as np
import matplotlib.pyplot as plt

size = 0.8
reps = 500
lambd = 0

data = np.loadtxt('../0b. DADOS/EMGsDataset.csv', delimiter=',')
C1, C2, C3, C4, C5 = 1, 2, 3, 4, 5
print(data)
results = []

[[0.000e+00 0.000e+00 9.050e+02 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 1.430e+03 ... 0.000e+00 3.941e+03 3.938e+03]
 [1.000e+00 1.000e+00 4.000e+00 ... 1.000e+00 2.000e+00 2.000e+00]]


In [32]:
X = np.concatenate((
  data[:2, data[2, :] == C1],
  data[:2, data[2, :] == C2],
  data[:2, data[2, :] == C3],
  data[:2, data[2, :] == C4],
  data[:2, data[2, :] == C5]
), axis=1)

p, N = X.shape

Y = np.concatenate((
  np.tile(C1 - 1, (10000, 1)),
  np.tile(C2 - 1, (10000, 1)),
  np.tile(C3 - 1, (10000, 1)),
  np.tile(C4 - 1, (10000, 1)),
  np.tile(C5 - 1, (10000, 1))
)).reshape(1, N)

In [33]:
results = []

for _ in range(reps):
  indices = np.arange(N)
  np.random.shuffle(indices)
  
  ind_treino = indices[:int(N*size)]
  ind_teste = indices[int(N*size):]
  
  X_treino, X_teste = X[:, ind_treino], X[:, ind_teste]
  Y_treino, Y_teste = Y[:, ind_treino], Y[:, ind_teste]
  
  X_group = []
  Cov_group = []
  
  for i in range(5):
    Xni = X_treino[:, Y_treino[0, :] == i]
    X_group.append(Xni)      
    Cov_group.append(np.cov(Xni))
  
  Means_group = []
  for i in X_group:
    Means_group.append(np.mean(i, axis=1).reshape(p, 1))
    
  Agr_matrix = np.zeros((p,p))
  for i in range(5):
    ni = X_group[i].shape[1]
    Agr_matrix += ni/N * Cov_group[i]
    
  Friedman_group = []
  for i in range(5):
    ni = X_group[i].shape[1]
    dividendo = (1 - lambd)*(ni * Cov_group[i])+(lambd * N * Agr_matrix)
    divisor = (1 - lambd) * ni + lambd * N
    Friedman_group.append(dividendo/divisor)

  inv_group = []    
  dets_group=[]
  for i in range(5):
    if lambd == 0:
      inv_group.append(np.linalg.pinv(Friedman_group[i]))
    else:
      inv_group.append(np.linalg.inv(Friedman_group[i]))

    dets_group.append(np.linalg.det(Friedman_group[i]))
    
  def discriminante(xn, mi, inv, det, lamda):
    if lamda != 1:
      t1 = -0.5 * np.log(det)
      t2 = -0.5 * (xn - mi).T @ inv @ (xn - mi)
      res = t1 + t2
    else:
      t1 = (xn - mi).T
      t2 = inv * (xn - mi)
      res = t1 * t2
    return res

  Y_pred = []
    
  for i in range(X_teste.shape[1]):
    discriminantes_matrix = []
    X_novo = X_teste[:, i].reshape(2, 1)
    for n in range(5):
      discriminantes_matrix.append(discriminante(X_novo, Means_group[n], inv_group[n], dets_group[n], lambd))
    Y_pred.append(np.argmax(discriminantes_matrix))
    
    
  Y_pred = np.array(Y_pred)
  Y_teste = Y_teste[0]
  results.append(np.mean(Y_pred  == Y_teste))

/tmp/ipykernel_54592/453832266.py:49: RuntimeWarning: divide by zero encountered in log
  t1 = -0.5 * np.log(det)


In [34]:
print("Média da Acurácia: ", np.mean(results))
print("Desvio Padrão: ", np.std(results))
print("Maior: ", np.max(results))
print("Menor: ", np.min(results))

Média da Acurácia:  0.20423999999999998
Desvio Padrão:  0.004209798094920936
Maior:  0.2098
Menor:  0.2
